# 言語モデルに、外部データを取り込む

OpenAIで用意している言語モデルは、ある日時までのデータで学習済みですが、今日出た最新の内容を学習しているわけではありません。また、利用者が保持している固有のデータを学習しているわけではありません。

LLMを使ったアプリケーションの大半は、モデルの学習セットには含まれないユーザ固有のデータが必要になります。
これを実現する主な方法は、RAG (Retrieval Augmented Generation) です。これは、外部のデータを取得して、プロンプト生成の際にこのデータを埋め込んでLLMに渡します。

LangChainには、RAGを実現するためのツールが備わっています。

## 準備

In [ ]:
## 利用するベースモデルのライブラリ (OpenAI) も別途インストールする
!pip install langchain==0.0.331 openai==0.28.1 python-dotenv cohere tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
## Googleドライブをマウント
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
## 環境変数設定
import dotenv
dotenv.load_dotenv('./drive/MyDrive/openai.env')

True

## LangChain Retrieval

RAG（Retrieval Augmented Generation）を実現するために必要なツールをまとめた、LangChainモジュールです。

![Retrieval](https://python.langchain.com/assets/images/data_connection-c42d68c3d092b85f50d08d4cc171fc25.jpg)

https://python.langchain.com/docs/modules/data_connection/

RAGを実現するための手順を簡単にみてみましょう。

### ドキュメントローダ (Load)

様々なソースからドキュメントをロードします。
LangChainは、100以上の異なるドキュメントローダを提供しています。
あらゆる種類のドキュメント（HTML／PDF／コード／等）をあらゆる場所（Amazon S3バケット／Webサイト／等）から読み込む機能が提供されています。

### ドキュメント変換 (Transform)

ドキュメント検索の重要なことは、ドキュメントの関連する部分のみをフェッチすることになります。そのために、検索用に最適化するための変換ステップが含まれます。
例えば、大きな文章を小さな文章に分割(chunking)することです。
LangChainは、これを行うためのアルゴリズムと、特定のドキュメントタイプ(コード／markdown／等)に最適化されたロジックを提供しています。

### テキスト埋め込み (Embed)

さらにドキュメント検索に重要なことは、テキストの埋め込み(Embedding)を作成することです。
Embeddingは、ざっくりに言うとテキスト文字列を数値ベクトルに置き換えます。
これをすることで、テキストの意味(semantec meaning)を捕らえ、類似した他のテキストを素早く効率的に検索することを可能にします。
LangChainは、25以上の異なるEmbedding Providerやメソッドとの統合を提供します。
LangChainは標準的なインターフェイスを提供し、簡単にモデル間の切り替えを可能にします。

### ベクター保存 (Vector Store)

テキスト文字列をEmbeddingすると、数値ベクトルに変換されます。
Embeddingで得られた値を効率よく保存・検索することを目的としたデータベースが必要になってきます。
LangChainは、50以上の異なるベクターストアとの統合を提供します。
LangChainは標準的なインターフェイスを公開しており、ベクターストアを簡単に交換することができます。

### 検索 (Retrieve)

ベクターデータがデータベースに格納されたら、それを取り出す必要があります。
LangChainは様々な検索アルゴリズムをサポートしています。
単純なセマンティック検索など、簡単に始められる基本的な方法をサポートしています。
しかし、パフォーマンスを向上させるために、これに加えて様々なアルゴリズムを追加しています。
以下がその例です：

- 親ドキュメント検索
    - 親ドキュメントごとに複数の埋め込みを作成することができ、より小さなチャンクを検索しつつ、より大きなコンテキストを返すことができます。
- セルフクエリ検索
    - ユーザーからの質問には、意味的なものだけでなく、メタデータフィルタとして表現するのが最適なロジックが含まれていることがよくあります。セルフクエリでは、クエリ内に存在する他のメタデータフィルタから、クエリのセマンティック部分を解析することができます。
- アンサンブル検索
    - 複数の異なるソースから、あるいは複数の異なるアルゴリズムを使って文書を取得したい場合があります。アンサンブル・リトリーバを使えば、これを簡単に行うことができます。


## ドキュメントローダ (Load)

シンプルな例を挙げます。ファイル内にあるテキストを読み込んで、1つのドキュメントオブジェクトとして配置します。


```python
from langchain.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()
```

今回はせっかくなので、PDFドキュメントを読み込んでみます。
試しに比較的大きめで、外部公開しても問題ないPDFドキュメントをGoogle Driveのマイドライブに格納しましょう。


In [ ]:
## PDFの読み込みに、今回はpypdfを使ってみます。
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
## 読み込みたいドキュメントを指定
## サンプルとして、外務省が公開しているSDGsの基礎資料のPDFを読み込ませてみる。
## https://www.mofa.go.jp/mofaj/gaiko/oda/sdgs/about/index.html

pdfdoc = './drive/MyDrive/sdgs_gaiyou_202310.pdf'


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdfdoc)
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(page_content='持続可能な開発目標（ SDGs）達成に\n向けて日本が果たす役割\n1ＳＤＧｓを通じて、豊かで活力ある未来を創る\n令和5年10月\n外務省国際協力局 地球規模課題総括課', metadata={'source': './drive/MyDrive/sdgs_gaiyou_202310.pdf', 'page': 0})

In [ ]:
pages[1]

Document(page_content='持続可能な開発目標（ SDGs）\n◼2015年9月の国連サミットで全会一致で採択。 「誰一人取り残さない」持続可能で多様性と包\n摂性のある社会 の実現のため、 2030年を年限とする 17の国際目標。 （その下に、 169のターゲット、\n231の指標が決められている。）\n先進国を含め、 全ての国が行動 普遍性\n人間の安全保障の理念を反映し\n「誰一人取り残さない 」 包摂性\n全てのステークホルダーが役割を 参画型\n社会・経済・環境に 統合的に取り組む 統合性\n定期的にフォローアップ 透明性\n\uf0752001年に国連で専門家間の議論を経て策定 。2000年に採択された 「国連ミレニアム宣言 」と、\n1990年代の主要な国際会議で採択された国際開発目標を統合したもの 。\n\uf075開発途上国向けの開発目標 として、 2015年を期限とする 8つの目標を設定。\n（①貧困・飢餓、②初等教育、③女性、④乳幼児、⑤妊産婦、⑥疾病、⑦環境、⑧連帯）前身：ミレニアム開発目標（ Millennium Development Goals: MDGs ）\n✓MDGsは一定の成果を達成。一方で、未達成の課題も残された。\n○極度の貧困半減（目標①）や HIV・マラリア対策（同⑥）等を達成。\n×乳幼児や妊産婦の死亡率削減（同④、⑤）は未達成。サブサハラアフリカ等で達成に遅れ。環境\n人権\n平和\n2', metadata={'source': './drive/MyDrive/sdgs_gaiyou_202310.pdf', 'page': 1})

ちなみに、今回はLangChainのドキュメントローダを利用しましたが、ファイル種別毎に用意されたライブラリを直接利用してテキスト抽出を行っても構いません。
個別のライブラリの方が多機能な部分があるため、そのような機能を使って抽出したい場合は、個別のライブラリを使う方が良いでしょう。


## ドキュメント変換 (Transform)

読み込んだドキュメントのテキストを元に、いくつかのチャンク(chunk)に分割します。
ここでは、LangChainの `RecursiveCharacterTextSplitter` を使います。

このテキスト分割ツールは、一般的なテキストに対して推奨されています。

そして、 `from_tiktoken_encoder()` を使って、トークン長を元にチャンクを分割します。
トークン長の推定は、OpenAIが公開している `tiktoken` というライブラリを用います。

https://github.com/openai/tiktoken


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    ## 適切なチャンクサイズは、ドキュメントによって異なるため、調整は必要
    ## 大きすぎると、回答時にいろんな箇所の情報が参照できない (トークン長に限りがあるため)
    ## 小さすぎると、回答の文脈がおかしくなる (途中で文章切れてしまうため)
    chunk_size = 200,
    chunk_overlap  = 0,
)

In [ ]:
## pageの区切りを '\n\n' に変換して、一つの文字列にしてchunkを生成
chunks = text_splitter.split_text('\n\n'.join([page.page_content for page in pages]))

In [ ]:
len(chunks)

46

In [ ]:
## 最初の5つ分のchunkを表示
chunks[:5]

['持続可能な開発目標（ SDGs）達成に\n向けて日本が果たす役割\n1ＳＤＧｓを通じて、豊かで活力ある未来を創る\n令和5年10月\n外務省国際協力局 地球規模課題総括課',
 '持続可能な開発目標（ SDGs）\n◼2015年9月の国連サミットで全会一致で採択。 「誰一人取り残さない」持続可能で多様性と包\n摂性のある社会 の実現のため、 2030年を年限とする 17の国際目標。 （その下に、 169のターゲット、\n231の指標が決められている。）',
 '先進国を含め、 全ての国が行動 普遍性\n人間の安全保障の理念を反映し\n「誰一人取り残さない 」 包摂性\n全てのステークホルダーが役割を 参画型\n社会・経済・環境に 統合的に取り組む 統合性\n定期的にフォローアップ 透明性',
 '\uf0752001年に国連で専門家間の議論を経て策定 。2000年に採択された 「国連ミレニアム宣言 」と、\n1990年代の主要な国際会議で採択された国際開発目標を統合したもの 。\n\uf075開発途上国向けの開発目標 として、 2015年を期限とする 8つの目標を設定。',
 '（①貧困・飢餓、②初等教育、③女性、④乳幼児、⑤妊産婦、⑥疾病、⑦環境、⑧連帯）前身：ミレニアム開発目標（ Millennium Development Goals: MDGs ）\n✓MDGsは一定の成果を達成。一方で、未達成の課題も残された。']

## テキスト埋め込み (Embedding)

Embedding可能なサービスは様々あります。
今回は、OpenAIが提供しているEmbeddings APIサービスをLangChainから使います。

実際には、この後のベクター保存の際にEmbeddingを兼ねて保存されますが、Embedding自体の振る舞いをここでは見てみます。

In [ ]:
import os
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    openai_api_key = os.getenv("OPENAI_API_KEY")  # set OpenAI API Key
)

embeddings = embeddings_model.embed_documents(chunks)

In [ ]:
len(embeddings), len(embeddings[0])

(46, 1536)

In [ ]:
## Embeddingした結果、最初のchunkの最初から5つの数値を表示
embeddings[0][:5]

[-0.012125153291959696,
 -0.02147120831104819,
 0.0020375601347753735,
 0.005812056922925895,
 -0.007675923532062142]

## ベクター保存 (Store)

Embeddingデータの保存と、ベクトル検索の実行を行います。
検索の際は、問い合わせクエリとなる文字列をEmbeddingして、保存されているデータとの類似度で検索をかけます。

![Vector Store](https://python.langchain.com/assets/images/vector_stores-9dc1ecb68c4cb446df110764c9cc07e0.jpg)

https://python.langchain.com/docs/modules/data_connection/vectorstores/

ベクター保存可能なサービスは、ここ最近急速に増えています。

今回は、Colaboratory上でも動作可能なローカルDBを利用します。



In [ ]:
## ベクターストアとして、Chromaを利用 (それ以外のライブラリは依存関係解消のため)
## 0.4.16だと、LangChainからの呼び出しでエラーになるので、一旦0.4.15に固定
## https://github.com/langchain-ai/langchain/issues/13051

!pip install chromadb==0.4.15 kaleido python-multipart "typing-extensions<4.6.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... d

In [ ]:
from langchain.vectorstores import Chroma

## データをベクターデータとして保存
db = Chroma.from_texts(texts=chunks, embedding=embeddings_model)

In [ ]:
## 試しに検索をかけてみる　(文字列版)
query = "日本における、SDGsアクションプラン2023は何ですか？"
docs = db.similarity_search(query)
print(docs[0].page_content)

「SDGsアクションプラン 2023」作成に当たっての基本的な考え方
•2023年はSDGsの「中間年」 。世界は歴史的な分水嶺に立ち、新たな挑戦に直面。新型コロナや
気候変動に加え、ロシアによるウクライナ侵略、食料やエネルギー安全保障などが相互に結びつき、


In [ ]:
## 試しに検索をかけてみる　(ベクター版)
query = "日本における、SDGsアクションプラン2023は何ですか？"
embedding_vector = embeddings_model.embed_query(query)  ## 問合せクエリ文字列をベクター化
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

「SDGsアクションプラン 2023」作成に当たっての基本的な考え方
•2023年はSDGsの「中間年」 。世界は歴史的な分水嶺に立ち、新たな挑戦に直面。新型コロナや
気候変動に加え、ロシアによるウクライナ侵略、食料やエネルギー安全保障などが相互に結びつき、


## 検索 (Retrieve)

実際の検索には、LangChain Retrieverを利用します。
LangChain Retrieverは、構造化されていないクエリを指定するとドキュメントを返すインタフェースになっています。ベクターストアはRetrieverのバックボーンとして利用します。

Retrieverは、LangChain Expression Language (LCEL) の基本構成要素であるRunnableインタフェースを実装しています。そのため、LCELでチェインを構成している場合、非常に便利です。
Runnableインタフェースは、invoke, ainvoke, stream, astream, batch, abatch, astream_log 呼び出しをサポートしています。

In [ ]:
## Retrieverインタフェース
retriever = db.as_retriever()

In [ ]:
## 検索
retrieved_docs = retriever.invoke(
    "日本における、SDGsアクションプラン2023は何ですか？"
)
print(retrieved_docs[0].page_content)

「SDGsアクションプラン 2023」作成に当たっての基本的な考え方
•2023年はSDGsの「中間年」 。世界は歴史的な分水嶺に立ち、新たな挑戦に直面。新型コロナや
気候変動に加え、ロシアによるウクライナ侵略、食料やエネルギー安全保障などが相互に結びつき、


ここまでで、RAGの事前準備ができました。

## ChatModelを使って、PDFの内容を質問しよう

ここからいよいよ、PDFの内容をプロンプトを含めた状態でChat Modelに質問を問い合わせてみましょう。


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain.chains import ConversationalRetrievalChain

## Setup Prompt Template
## 今回はデフォルトのテンプレートを使うので、カスタム設定しないでおく。

## Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Setup Chat Model
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    openai_api_key = os.getenv("OPENAI_API_KEY"),  # set OpenAI API Key
    temperature = 0.7,
    # max_tokens = 255,
)

## Setup Retriever
retriever = db.as_retriever(
    search_type='similarity',  ## 検索タイプ: 'similarity' / 'similarity_score_threshold' / 'mmr' (Maximum Marginal Relevance)
    search_kwargs={"k": 5},  ## 検索取得数: default: 4
)

## Setup Chain
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=True,
)


In [ ]:
## 会話開始
query = "日本における、SDGsアクションプラン2023は何ですか？"
res = qa({"question": query})
res



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
「SDGsアクションプラン 2023」作成に当たっての基本的な考え方
•2023年はSDGsの「中間年」 。世界は歴史的な分水嶺に立ち、新たな挑戦に直面。新型コロナや
気候変動に加え、ロシアによるウクライナ侵略、食料やエネルギー安全保障などが相互に結びつき、

定している。また、 SDGs達成に資する優れた取組を行う企業・団体等を 「ジャパン SDGsアワード」 を通じて表彰して
いる。
32023年9月
SDGサミット開催
2023年中（予定）
SDGs実施指針の改定2023年はSDGsの「中間年」。
9月には 4年に1度となる SDGサミットが開催。

される。こうした機会を最大限活用し日本の取組を発信する。SDGsアクションプラン 2023：概要
4【ポイント】
◆「人への投資 」、科学技術・イノベーション への投資、 スタートアップ への
投資、グリーントランスフォーメーション（ GX）、デジタルトランスフォー

行政、民間企業、有識者、ＮＧＯ等、
広範な関係者が意見交換を行う。
日本が 2030アジェンダを実施し、
2030年までに国内外において
SDGsを達成するための中長期的な国家戦略。
2019年9月に開催された SDGサミットと、
日本国内における SDGsの取組進展を踏まえて改定。

本年9月に開催された SDGサミットと日本国内における SDGsの取組進展
を踏まえ、 SDGs実施指針の改定を予定。
Human: 日本における、SDGsアクションプラン2023は何ですか？

> Finished chain.

> Finished chain.


{'question': '日本における、SDGsアクションプラン2023は何ですか？',
 'chat_history': [HumanMessage(content='日本における、SDGsアクションプラン2023は何ですか？'),
  AIMessage(content='SDGsアクションプラン2023は、日本が2030年までに国内外でSDGsを達成するための中長期的な国家戦略です。具体的な内容は上記の文脈では明記されていませんが、2023年にはSDGsの中間年となるため、日本は様々な課題に直面し、新たな挑戦に取り組むことが期待されています。SDGsアクションプラン2023の具体的な内容については、さらなる情報が必要です。')],
 'answer': 'SDGsアクションプラン2023は、日本が2030年までに国内外でSDGsを達成するための中長期的な国家戦略です。具体的な内容は上記の文脈では明記されていませんが、2023年にはSDGsの中間年となるため、日本は様々な課題に直面し、新たな挑戦に取り組むことが期待されています。SDGsアクションプラン2023の具体的な内容については、さらなる情報が必要です。'}

より詳細なRAGの手法が知りたい場合は、以下のサイト等を参考にしましょう。

- [Retrieval-augmented generation (RAG) | 🦜️🔗 Langchain](https://python.langchain.com/docs/use_cases/question_answering/)
- [Retrieval | 🦜️🔗 Langchain](https://python.langchain.com/docs/modules/data_connection/)

## RAGアプリケーションを作る際の検討事項

大量のドキュメントの中から回答を得るような、大規模なRAGアプリケーションを構築する場合、いくつか検討する事項があります。


### 欲しい情報がどのドキュメントにあるのか？

RAGでは、既存のドキュメントの内容をプロンプト内に入れることで、その情報を元に回答してくれます。
ドキュメントが大量にある場合、まず既存のドキュメントの検索をどのように行うかを検討する必要があります。

今回のサンプルコードでは、ローカルメモリ上に構築したベクターストアにドキュメントのEmbedding情報を格納し、検索することを行いました。ローカルメモリ上では情報の永続化ができないので、永続化できないといけません。
今回使用した「Chroma」は、永続化オプションがあります。
しかし、ローカルマシン上に格納されるので、負荷が高くなった時のスケールアウトが難しくなります。

実際のアプリケーションでは、ベクターストアはアプリケーションとは別の外部にあることが望ましいです。

例えば、PostgreSQLにpg_vectorという拡張モジュールがあり、これを使うことでPostgreSQL上でベクトル管理・検索が可能になります。Amazon RDSにも対応しています。

- 参考: [Amazon RDS for PostgreSQLがpgvectorモジュールに対応しベクトル検索できるようになりました | DevelopersIO](https://dev.classmethod.jp/articles/amazon-rds-postgresql-pgvector-embedding/)

また、近年ベクターストアサービスがたくさん出てきています。

Momento Vector Indexや、Cloudflare Vectorizeといった、Webアプリケーション等で利用しやすいサービスもあります。

- 参考: [サーバレスなVector Database、Momento Vector Indexの紹介 | DevelopersIO](https://dev.classmethod.jp/articles/momento-vector-index/)
- 参考: [Cloudflare Vectorize · Vectorize](https://developers.cloudflare.com/vectorize)

### ドキュメント検索サービスを使う方法

ベクターストア以外にドキュメント検索する方法としては、ドキュメント検索サービスを使う方法があります。
AWSであれば、Amazon Kendra、Google Cloudであれば、Enterprize Searchがあります。
AWSはAmazon KendraとOpenAIを活用したサンプルチャットアプリケーションを公開しています。

- 参考: [Amazon Kendra と OpenAI により最新の AWS ユーザーガイドに基づいて回答するチャットアプリケーションのサンプルを試してみた | DevelopersIO](https://dev.classmethod.jp/articles/using-amazon-kendra-langchain-extensions/)
- 参考: [【Google Cloud】Enterprise Searchで社内ドキュメントを検索してみた | DevelopersIO](https://dev.classmethod.jp/articles/try-google-cloud-enterprise-search/)

ドキュメント検索サービスだけ使っても、言語モデルからの応答は得られません。
この場合、ドキュメント検索サービスの検索結果の上位のドキュメントから、必要な情報を抽出して、言語モデルのプロンプトに入れて回答を得る必要があります。

この方法では、大量のドキュメントを事前にベクターストアに格納する必要がなく、引用元のドキュメントもわかるという利点があります。

### 参考構成

![AWS_RAG](https://devio2023-media.developers.io/wp-content/uploads/2023/10/Untitled.png)

https://dev.classmethod.jp/articles/implement-rag-with-aws-services/

![GCP_RAG](https://devio2023-media.developers.io/wp-content/uploads/2023/09/Picture3-1-640x203.png)

https://dev.classmethod.jp/articles/improve-work-efficiency-with-generateive-ai-chatbot-using-rag/

![GCP_Enterprize_RAG](https://devio2023-media.developers.io/wp-content/uploads/2023/09/Picture1-1-1536x944.png)

https://dev.classmethod.jp/articles/qa-with-google-cloud-enterprise-search-and-retrieve-read-compose-rag/
